In [1]:
import importlib
import Failure_API.src.wrapper_api.wrappers.noise_wrapper as nw
import Failure_API.src.wrapper_api.failure_apis.sharedobs_wrapper as sh
import Failure_API.src.wrapper_api.wrappers.communication_wrapper as cmw
import Failure_API.src.wrapper_api.noise_models.base_noise_model as nm
import Failure_API.src.wrapper_api.connectivity_patterns.base_communication_model as cm
import Failure_API.src.wrapper_api.connectivity_patterns.active_communication as ac

importlib.reload(nw)
importlib.reload(sh)
importlib.reload(cm)
importlib.reload(cmw)
importlib.reload(nm)
importlib.reload(cm)
importlib.reload(ac)

from Failure_API.src.wrapper_api.failure_apis.sharedobs_wrapper import SharedObsWrapper
from Failure_API.src.wrapper_api.failure_apis import CommunicationWrapper, NoiseWrapper

from Failure_API.src.wrapper_api.noise_models.base_noise_model import GaussianNoise
from Failure_API.src.wrapper_api.connectivity_patterns.base_communication_model import ProbabilisticModel
import pandas as pd
from pettingzoo.mpe import simple_spread_v3

In [14]:
env = simple_spread_v3.env()
env.reset()

env = SharedObsWrapper(env)
env = CommunicationWrapper(env, failure_models=[ProbabilisticModel(agent_ids=env.possible_agents, failure_prob=0.5)])
env = NoiseWrapper(env, noise_model=GaussianNoise(mean=0.3, std=0.05))

columns = [ "Viewing Agent", "Other_Agent", "self_velX", "self_velY", "self_posX", "self_posY", "l_rel_p1X", "l-rel_p1Y", "l_rel_p2X", "l_rel_p2Y", "l_rel_p3X", "l_rel_p3Y", "o_a1x", "o_a1y", "o_2x", "o_a2y", "c1", "c2", "c3", "c4"]
records = []

for agent in env.agent_iter():
    obs, rew, term, trunc, info = env.last()

    if term or trunc:
        action = None
    else:
        action = env.action_space(agent).sample()

    if obs is not None:  # ✅ Check here
        print(f"{agent} sees:")
        for other, data in obs.items():
            print(f"  {other} -> {data}")

        record = [agent, other] + data.tolist()
        records.append(record)
    else:
        print(f"{agent} is inactive (no observation).")

    print("----")

    if all(env.terminations.values()) or all(env.truncations.values()):
        break

    env.step(action)



agent_0 sees:
  agent_0 -> [ 0.          0.          0.51961344 -0.2538111  -0.00973639  0.35805655
 -0.7121672   0.99977934 -1.0703784   0.30407372 -1.1395767   1.0771048
 -0.25641266 -0.34807926  0.          0.          0.          0.        ]
  agent_1 -> [ 0.29222784  0.27664042 -0.30416289  1.14056064  1.32279666 -0.4594375
  0.7685794   0.263447    0.38956599 -0.55983806  1.38970272 -0.79731812
  1.22729868 -1.12143771  0.38306528  0.2309284   0.34303814  0.3001011 ]
  agent_2 -> [ 0.3082673   0.30057848  0.58211704 -0.27242453  0.57453178  1.01098658
 -0.1451096   1.70907792 -0.60054322  0.94753268  0.55852333  0.65301216
 -0.5157452   1.76608471  0.36838137  0.38376418  0.33018641  0.29223959]
----
agent_1 sees:
  agent_0 -> [0.30554896 0.23920789 0.30579683 0.24731689 0.28369114 0.29738746
 0.26943093 0.28949199 0.22940485 0.25538702 0.267149   0.32167547
 0.32144843 0.30537444 0.33458008 0.35931269 0.26500038 0.32185814]
  agent_1 -> [ 0.          0.         -0.6199632   0.82

In [15]:


for other, data in obs.items():
    record = [agent, other] + data.tolist()
    records.append(record)
df = pd.DataFrame(records, columns=columns)
df.head()

,Viewing Agent,Other_Agent,self_velX,self_velY,self_posX,self_posY,l_rel_p1X,l-rel_p1Y,l_rel_p2X,l_rel_p2Y,l_rel_p3X,l_rel_p3Y,o_a1x,o_a1y,o_2x,o_a2y,c1,c2,c3,c4
0,agent_0,agent_2,0.308267,0.300578,0.582117,-0.272425,0.574532,1.010987,-0.145110,1.709078,-0.600543,0.947533,0.558523,0.653012,-0.515745,1.766085,0.368381,0.383764,0.330186,0.292240
1,agent_1,agent_2,0.285838,0.293311,0.244924,0.293774,0.276704,0.273420,0.235012,0.246820,0.191802,0.324607,0.268770,0.314398,0.319086,0.373828,0.287574,0.267579,0.326739,0.259157
2,agent_2,agent_2,0.000000,0.000000,0.263201,-0.601890,0.246676,0.706136,-0.455755,1.347859,-0.813966,0.652153,0.256413,0.348079,-0.883164,1.425184,0.000000,0.000000,0.000000,0.000000
3,agent_0,agent_2,-0.180010,0.275267,0.548105,-0.285260,0.505729,1.065485,-0.103093,1.667816,-0.396960,0.881354,0.501176,0.636537,-0.543704,1.790323,0.335147,0.319370,0.250256,0.328011
4,agent_1,agent_2,0.256770,0.355513,0.284284,0.242939,0.325652,0.311864,0.308358,0.282875,0.299686,0.271724,0.296083,0.312088,0.296206,0.307885,0.247488,0.273044,0.359154,0.379138
